## Orchestrations: Pre-built Multi-Agent Patterns

In the previous notebook, you learned to build workflows from scratch using executors and edges. **Orchestrations** are pre-built workflow patterns that let you quickly create multi-agent systems by simply plugging in your agents.

In this notebook, we'll cover three orchestration patterns:

- **Sequential** - Agents work in a pipeline, each building on the previous output
- **Concurrent** - Agents work in parallel on the same task, results are aggregated
- **Group Chat** - Agents collaborate in a managed conversation with iterative refinement

We'll use a common theme throughout: **launching a new product**. This will help you see how each pattern fits different stages of a real workflow.

### Setup

First, let's validate our environment and create the Azure OpenAI client.

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from workshop_utils import validate_env

validate_env()

In [ ]:
import os
from agent_framework.azure import AzureOpenAIChatClient

chat_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

print("Client created successfully!")

### When to Use Each Pattern

Before diving in, here's a quick decision guide:

| Pattern | Use When | Example |
|---------|----------|--------|
| **Sequential** | Each step builds on the previous | Draft → Review → Polish |
| **Concurrent** | Tasks are independent, need multiple perspectives | Research + Legal + Marketing analysis |
| **Group Chat** | Iterative refinement through collaboration | Writer ↔ Editor feedback loop |

The Agent Framework also provides **Handoff** (dynamic routing) and **Magentic** (manager-driven) patterns - see the [documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/overview) for details.

## Sequential Orchestration

In sequential orchestration, agents work in a pipeline - each agent processes the task in turn, and its output becomes input for the next agent.

**Scenario**: A copywriter drafts marketing content, then an editor reviews and improves it.

Let's start by creating two agents:

In [ ]:
# Create two agents for a draft -> review pipeline
copywriter = chat_client.as_agent(
    name="copywriter",
    instructions="You are a creative copywriter. Write compelling, concise marketing copy for product launches.",
)

editor = chat_client.as_agent(
    name="editor",
    instructions="You are an editor. Review the copywriter's draft and improve clarity, impact, and grammar. Provide the revised version.",
)

Now we build the workflow with `SequentialBuilder` and run it:

In [ ]:
from agent_framework import SequentialBuilder, ChatMessage, Role, WorkflowOutputEvent

# Build the pipeline: copywriter -> editor
workflow = SequentialBuilder().participants([copywriter, editor]).build()

# Run and capture the final output
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream("Write a tagline and short description for our new eco-friendly water bottle."):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# Print the conversation showing each agent's contribution
if output_evt:
    print("===== Sequential Pipeline Output =====")
    messages: list[ChatMessage] = output_evt.data
    for msg in messages:
        name = msg.author_name or ("user" if msg.role == Role.USER else "assistant")
        print(f"\n[{name}]\n{msg.text}")
        print("-" * 50)

## Concurrent Orchestration

In concurrent orchestration, multiple agents work on the same task **in parallel**. Each agent processes the input independently, and results are aggregated at the end.

**Scenario**: Before launching a product, we need analysis from three teams simultaneously - research, marketing, and legal.

Let's create three specialist agents:

In [ ]:
# Create three domain experts who will work in parallel
researcher = chat_client.as_agent(
    name="researcher",
    instructions="You're a market researcher. Provide concise insights on opportunities and risks.",
)

marketer = chat_client.as_agent(
    name="marketer",
    instructions="You're a marketing strategist. Craft a compelling value proposition and target audience.",
)

legal_reviewer = chat_client.as_agent(
    name="legal",
    instructions="You're a legal/compliance reviewer. Highlight regulatory concerns and required disclaimers.",
)

Now build with `ConcurrentBuilder` - all agents receive the same prompt and run simultaneously:

In [ ]:
from agent_framework import ConcurrentBuilder, ChatMessage, WorkflowOutputEvent

# Build concurrent workflow
workflow = ConcurrentBuilder().participants([researcher, marketer, legal_reviewer]).build()

# Run - all agents process the same prompt in parallel
prompt = "We're launching a budget-friendly electric bike for urban commuters. Analyze from your perspective."
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# Print each agent's independent analysis
if output_evt:
    print("===== Concurrent Analysis Results =====")
    messages: list[ChatMessage] = output_evt.data
    for msg in messages:
        if msg.author_name:
            print(f"\n{'='*50}")
            print(f"[{msg.author_name.upper()}]")
            print(f"{'='*50}")
            print(msg.text)

## Group Chat Orchestration

In group chat orchestration, agents collaborate in a **managed conversation**. Unlike sequential (one direction) or concurrent (independent), group chat enables iterative back-and-forth where agents build on each other's responses.

Key components:
- **Speaker selector**: Determines who speaks next (custom function or AI-managed)
- **Termination condition**: Defines when the conversation ends

**Scenario**: A writer creates product launch content, an editor refines it, and a fact-checker verifies claims - iterating until the content is ready.

First, let's create our three collaborating agents:

In [ ]:
# Create three agents for iterative content collaboration
content_writer = chat_client.as_agent(
    name="Writer",
    instructions="""You are a creative content writer for product launches.
    Write engaging copy and incorporate feedback from the editor and fact-checker.
    Keep responses concise (2-3 paragraphs max).""",
)

content_editor = chat_client.as_agent(
    name="Editor",
    instructions="""You are a content editor focused on clarity and impact.
    Review the writer's content and suggest specific improvements.
    Be constructive and concise.""",
)

fact_checker = chat_client.as_agent(
    name="FactChecker",
    instructions="""You verify claims and suggest accurate alternatives.
    Flag any unsubstantiated claims and recommend evidence-based revisions.
    Be brief and specific.""",
)

Now we build the group chat with a custom speaker selector. The selector receives a `GroupChatState` object with the conversation history and returns the next speaker's name:

In [ ]:
from agent_framework import (
    ChatMessage,
    GroupChatBuilder,
    GroupChatState,
    WorkflowOutputEvent,
    AgentRunUpdateEvent,
)

def select_next_speaker(state: GroupChatState) -> str:
    """Round-robin: Writer -> Editor -> FactChecker -> Writer -> ..."""
    order = ["Writer", "Editor", "FactChecker"]
    return order[state.current_round % len(order)]

# Build the group chat workflow
workflow = (
    GroupChatBuilder()
    .with_select_speaker_func(select_next_speaker, orchestrator_name="Orchestrator")
    .participants([content_writer, content_editor, fact_checker])
    .with_termination_condition(lambda conv: len(conv) >= 5)  # Stop after 5 turns
    .build()
)

# Run with streaming to see the conversation unfold
task = "Write a short product description for our new eco-friendly water bottle that keeps drinks cold for 24 hours."
print(f"TASK: {task}\n")
print("=" * 60)

async for event in workflow.run_stream(ChatMessage(role="user", text=task)):
    if isinstance(event, AgentRunUpdateEvent):
        print(event.data, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print("\n" + "=" * 60)
        print("Collaboration complete!")

<details>
<summary><strong>GroupChatState Reference</strong></summary>

The `GroupChatState` object passed to your selector has these attributes:

| Attribute | Type | Description |
|-----------|------|-------------|
| `participants` | `dict[str, str]` | Agent names mapped to descriptions |
| `conversation` | `list[ChatMessage]` | Full conversation history |
| `current_round` | `int` | Number of selection rounds so far |

You can also use `.with_agent_orchestrator(agent)` instead of a custom function to let an AI agent decide who speaks next.

</details>

### Exercise: Build a Product Launch Review System

**Scenario**: Your company is launching a new AI-powered fitness app. Before launch, you need multiple teams to review and provide feedback.

| Agent | Role | Focus |
|-------|------|-------|
| `product_manager` | Product Strategy | Feature completeness, market fit, user value |
| `security_reviewer` | Security & Privacy | Data handling, GDPR compliance, vulnerabilities |
| `ux_designer` | User Experience | Usability, accessibility, design consistency |

**Your Task**: Choose an orchestration pattern and implement the review system.

**Hints**:
- Consider: Should reviews happen in sequence (each building on the last) or in parallel (independent perspectives)?
- For parallel reviews, use `ConcurrentBuilder`
- For sequential reviews with synthesis, use `SequentialBuilder`
- For iterative discussion, use `GroupChatBuilder`

In [ ]:
# TODO: Create three agents for the product launch review
product_manager = chat_client.as_agent(
    name="product_manager",
    instructions="""You are a product manager reviewing a product launch.
    Focus on: feature completeness, market fit, and user value proposition.
    Provide specific, actionable feedback.""",
)

security_reviewer = chat_client.as_agent(
    name="security_reviewer",
    instructions="""You are a security and privacy reviewer.
    Focus on: data handling practices, GDPR/privacy compliance, potential vulnerabilities.
    Flag any concerns with severity ratings.""",
)

ux_designer = chat_client.as_agent(
    name="ux_designer",
    instructions="""You are a UX designer reviewing a product.
    Focus on: usability, accessibility, design consistency, user journey.
    Suggest specific improvements.""",
)

# TODO: Choose and build your orchestration
# Option 1: ConcurrentBuilder for parallel independent reviews
# Option 2: SequentialBuilder for reviews that build on each other
# Option 3: GroupChatBuilder for iterative discussion

# workflow = ...

# TODO: Run the workflow with this prompt
prompt = """Review our new AI fitness app 'FitAI' for launch readiness:
- Tracks workouts using phone sensors
- Uses AI to generate personalized workout plans
- Stores user health data in the cloud
- Integrates with social media for sharing achievements
- Subscription model: $9.99/month"""

# result = await workflow.run(prompt)
# print(result)

<details>
<summary><strong>Click to reveal solution</strong></summary>

```python
from agent_framework import ConcurrentBuilder, ChatMessage, WorkflowOutputEvent

# Create agents
product_manager = chat_client.as_agent(
    name="product_manager",
    instructions="""You are a product manager reviewing a product launch.
    Focus on: feature completeness, market fit, and user value proposition.
    Provide specific, actionable feedback.""",
)

security_reviewer = chat_client.as_agent(
    name="security_reviewer",
    instructions="""You are a security and privacy reviewer.
    Focus on: data handling practices, GDPR/privacy compliance, potential vulnerabilities.
    Flag any concerns with severity ratings.""",
)

ux_designer = chat_client.as_agent(
    name="ux_designer",
    instructions="""You are a UX designer reviewing a product.
    Focus on: usability, accessibility, design consistency, user journey.
    Suggest specific improvements.""",
)

# Concurrent is a good choice here - each reviewer provides independent perspective
workflow = ConcurrentBuilder().participants([product_manager, security_reviewer, ux_designer]).build()

prompt = """Review our new AI fitness app 'FitAI' for launch readiness:
- Tracks workouts using phone sensors
- Uses AI to generate personalized workout plans
- Stores user health data in the cloud
- Integrates with social media for sharing achievements
- Subscription model: $9.99/month"""

# Run with streaming and capture output
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# Print each reviewer's feedback
if output_evt:
    messages: list[ChatMessage] = output_evt.data
    for msg in messages:
        if msg.author_name:
            print(f"\n{'='*60}")
            print(f"[{msg.author_name}]")
            print(f"{'='*60}")
            print(msg.text)
```

</details>

**Bonus Challenge**: Extend your solution to add a `synthesizer` agent that takes all the reviews and creates a consolidated launch readiness report. Consider using Sequential orchestration to chain the concurrent reviews with the synthesis step.

---
## Summary & Recap

In this notebook, you learned about pre-built orchestration patterns in the Agent Framework:

### Key Concepts

| Orchestration | Description |
|---------------|-------------|
| **Sequential** | Agents execute one after another in a pipeline, each building on the previous output |
| **Concurrent** | Multiple agents work on the same task in parallel, results are aggregated |
| **Group Chat** | Collaborative conversation with a chat manager controlling speaker selection |
| **Handoff** | Dynamic control transfer between agents based on context |
| **Magentic** | Manager-driven approach where agents pull tasks based on evolving context |

### What You Built

1. **Sequential Pipeline** - Copywriter → Editor for drafting and refining marketing copy
2. **Concurrent Workflow** - Researcher, Marketer, and Legal agents analyzing a product launch in parallel
3. **Group Chat** - Writer, Editor, and FactChecker collaborating iteratively on content

### Key Patterns

```python
# Sequential orchestration
workflow = SequentialBuilder().participants([agent1, agent2]).build()

# Concurrent orchestration
workflow = ConcurrentBuilder().participants([agent1, agent2, agent3]).build()

# Group chat with custom speaker selection
def select_speaker(state: GroupChatState) -> str:
    # Custom logic to determine next speaker
    return next_agent_name

workflow = (
    GroupChatBuilder()
    .with_select_speaker_func(select_speaker, orchestrator_name=\"Orchestrator\")
    .participants([agent1, agent2, agent3])
    .with_termination_condition(lambda conversation: len(conversation) >= 6)
    .build()
)
```

### Chapter 2 Complete!

You now have the skills to build sophisticated multi-agent systems with:
- Custom workflows with executors and edges (02.1)
- Agents as workflow components (02.1)
- Pre-built orchestration patterns (02.2)
- Event-driven observability and streaming (02.1, 02.2)

In **Chapter 3**, you'll learn about **RAG (Retrieval-Augmented Generation)** to give your agents access to external knowledge!